##### **Mouting Google Drive**


In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# **Preprocessing**

##### **Loading in the Data, removing redundant columns and rows with NULL values**




In [0]:
import pandas as pd
ct = pd.read_csv('/content/gdrive/My Drive/data3.csv')
ct = ct.drop(columns=['ID', 'Zip Code Boundaries', '2010 Census Tracts', 'Florida Census Zip Codes', 'Commission Districts', 'Report Hour of Day', 'Report Day of Week', 'City', 'State', 'Report Date', 'Location', ':@computed_region_9cfm_spy5', ':@computed_region_43jd_v64e', ':@computed_region_ndi2_bfht'])
ct = ct.dropna()
ct

,Incident Type,Offense Date,Offense Hour of Day,Offense Day of Week,Address,Latitude,Longitude,Gainesville Police Districts
0,Stalking (simple),11/02/2019 10:44:57 AM,10,Saturday,3800 BLK SW 34TH ST,29.618756,-82.372637,3.0
1,Disturbance,11/02/2019 09:25:58 AM,9,Saturday,3800 BLK SW ARCHER RD,29.620465,-82.380355,12.0
2,Suspicious Incident,10/31/2019 09:00:37 PM,21,Thursday,2000 BLK NW 20TH LN,29.669724,-82.348361,4.0
3,Domestic Disturbance,11/02/2019 11:30:00 AM,11,Saturday,4400 BLK SW 20TH AVE,29.636340,-82.389375,12.0
4,Theft Petit - Retail,11/02/2019 12:45:00 PM,12,Saturday,500 BLK N MAIN ST,29.655751,-82.325024,1.0
...,...,...,...,...,...,...,...,...
151375,Suspicious Incident,08/04/2018 08:30:14 PM,20,Saturday,6400 BLK W NEWBERRY RD,29.659701,-82.411853,10.0
151376,Drug Violation (sid),01/29/2018 01:00:32 PM,13,Monday,BLK MM 375 NORTHBOUND,29.651950,-82.322780,1.0
151377,Battery (simple),01/15/2019 08:29:40 PM,20,Tuesday,1100 BLK E UNIVERSITY AVE,29.651993,-82.310770,2.0
151378,Damage to Property,10/14/2017 12:00:00 AM,0,Saturday,1200 BLK W UNIVERSITY AVE,29.652073,-82.337100,1.0


##### **Extracting the Day, Month and Year values from the Offense Date column and adding new columns for the extracted values**

In [0]:
import datetime

def day(a):
  a=a[0:10]
  datee = datetime.datetime.strptime(a, "%m/%d/%Y")
  return datee.day

def month(a):
  a=a[0:10]
  datee = datetime.datetime.strptime(a, "%m/%d/%Y")
  return datee.month

def year(a):
  a=a[0:10]
  datee = datetime.datetime.strptime(a, "%m/%d/%Y")
  return datee.year

ct['Day'] = ct.apply(lambda row: day(row['Offense Date']), axis=1)
ct['Month'] = ct.apply(lambda row: month(row['Offense Date']), axis=1)
ct['Year'] = ct.apply(lambda row: year(row['Offense Date']), axis=1)
ct

,Incident Type,Offense Date,Offense Hour of Day,Offense Day of Week,Address,Latitude,Longitude,Gainesville Police Districts,Day,Month,Year
0,Stalking (simple),11/02/2019 10:44:57 AM,10,Saturday,3800 BLK SW 34TH ST,29.618756,-82.372637,3.0,2,11,2019
1,Disturbance,11/02/2019 09:25:58 AM,9,Saturday,3800 BLK SW ARCHER RD,29.620465,-82.380355,12.0,2,11,2019
2,Suspicious Incident,10/31/2019 09:00:37 PM,21,Thursday,2000 BLK NW 20TH LN,29.669724,-82.348361,4.0,31,10,2019
3,Domestic Disturbance,11/02/2019 11:30:00 AM,11,Saturday,4400 BLK SW 20TH AVE,29.636340,-82.389375,12.0,2,11,2019
4,Theft Petit - Retail,11/02/2019 12:45:00 PM,12,Saturday,500 BLK N MAIN ST,29.655751,-82.325024,1.0,2,11,2019
...,...,...,...,...,...,...,...,...,...,...,...
151375,Suspicious Incident,08/04/2018 08:30:14 PM,20,Saturday,6400 BLK W NEWBERRY RD,29.659701,-82.411853,10.0,4,8,2018
151376,Drug Violation (sid),01/29/2018 01:00:32 PM,13,Monday,BLK MM 375 NORTHBOUND,29.651950,-82.322780,1.0,29,1,2018
151377,Battery (simple),01/15/2019 08:29:40 PM,20,Tuesday,1100 BLK E UNIVERSITY AVE,29.651993,-82.310770,2.0,15,1,2019
151378,Damage to Property,10/14/2017 12:00:00 AM,0,Saturday,1200 BLK W UNIVERSITY AVE,29.652073,-82.337100,1.0,14,10,2017


##### **Dropping the Offense Date column after completed extraction**

In [0]:
ct = ct.drop(columns='Offense Date')
ct

,Incident Type,Offense Hour of Day,Offense Day of Week,Address,Latitude,Longitude,Gainesville Police Districts,Day,Month,Year
0,Stalking (simple),10,Saturday,3800 BLK SW 34TH ST,29.618756,-82.372637,3.0,2,11,2019
1,Disturbance,9,Saturday,3800 BLK SW ARCHER RD,29.620465,-82.380355,12.0,2,11,2019
2,Suspicious Incident,21,Thursday,2000 BLK NW 20TH LN,29.669724,-82.348361,4.0,31,10,2019
3,Domestic Disturbance,11,Saturday,4400 BLK SW 20TH AVE,29.636340,-82.389375,12.0,2,11,2019
4,Theft Petit - Retail,12,Saturday,500 BLK N MAIN ST,29.655751,-82.325024,1.0,2,11,2019
...,...,...,...,...,...,...,...,...,...,...
151375,Suspicious Incident,20,Saturday,6400 BLK W NEWBERRY RD,29.659701,-82.411853,10.0,4,8,2018
151376,Drug Violation (sid),13,Monday,BLK MM 375 NORTHBOUND,29.651950,-82.322780,1.0,29,1,2018
151377,Battery (simple),20,Tuesday,1100 BLK E UNIVERSITY AVE,29.651993,-82.310770,2.0,15,1,2019
151378,Damage to Property,0,Saturday,1200 BLK W UNIVERSITY AVE,29.652073,-82.337100,1.0,14,10,2017


##### **Exporting preprocessed data file**

In [0]:
from google.colab import files
ct.to_csv('cleandata.csv') 
files.download('cleandata.csv')

KeyboardInterrupt: ignored

#**Classification**

##### **Encoding the attributes with String data type**

In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
ct['Incident Type'] = le.fit_transform(ct['Incident Type'].astype(str))
ct['Incident Type'] = le.fit_transform(ct['Incident Type'])

le2 = preprocessing.LabelEncoder()
ct['Offense Day of Week'] = le2.fit_transform(ct['Offense Day of Week'].astype(str))
ct['Offense Day of Week'] = le2.fit_transform(ct['Offense Day of Week'])

le3 = preprocessing.LabelEncoder()
ct['Address'] = le2.fit_transform(ct['Address'].astype(str))
ct['Address'] = le2.fit_transform(ct['Address'])

ct['Incident Type'].nunique()

218

##### **Splitting dataset into train and test sets**

In [0]:
from sklearn.model_selection import train_test_split
y = ct['Incident Type']
ct = ct.drop(columns= 'Incident Type')

X_train, X_test, y_train, y_test = train_test_split(ct, y, test_size=0.20)

print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)


(120209, 9) (120209,)
(30053, 9) (30053,)


##### **SGD**

Fit

In [0]:
%%time
from sklearn import model_selection, linear_model
clf = linear_model.SGDClassifier(loss='log')
clf.fit(X_train, y_train)

CPU times: user 6min 36s, sys: 52.6 ms, total: 6min 36s
Wall time: 6min 36s


Accuracy

In [0]:
clf.score(X_test, y_test)

0.03660200312780754

Confusion matrix

In [0]:
import numpy as np
np.set_printoptions(threshold=np.inf, linewidth = 2026)
from sklearn.metrics import confusion_matrix
y_pred = clf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    6,    0,    0,    0,    0,    0,    0,    6,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    6,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    2,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   

##### **KNN**

Fit

In [0]:
%%time
from sklearn import neighbors, metrics
clf2 = neighbors.KNeighborsClassifier()
clf2.fit(X_train, y_train)

CPU times: user 161 ms, sys: 2.98 ms, total: 164 ms
Wall time: 228 ms


Accuracy

In [0]:
clf2.score(X_test, y_test)

0.11083752038066083

In [0]:
y_pred = clf2.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   0,    0,    0,    0,    0,    0,    1,    0,    4,    0,    0,    1,    2,    0,    2,    1,    0,    0,    0,    0,    1,    9,    0,    0,    1,    0,    0,    0,    3,    4,    0,    0,    0,    1,    0,    0,    0,    0,    3,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    1,    0,    0,    1,    0,    0,    0,    0,    0,    0,    0,    2,    0,    0,    0,    1,    0,    0,    0,    0,    1,    0,    0,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    2,    0,    1,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   

##### **Random Forest**

Fit

In [0]:
%%time
from sklearn import ensemble
clf3 = ensemble.RandomForestClassifier()
clf3.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 7.53 s, sys: 877 ms, total: 8.41 s
Wall time: 8.48 s


Accuracy

In [0]:
clf3.score(X_test, y_test)

0.13296509499883538

Confusion matrix

In [0]:
y_pred = clf3.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   0,    0,    0,    0,    0,    0,    0,    0,    2,    0,    1,    0,    1,    0,    0,    0,    0,    0,    0,    0,    0,    6,    0,    0,    0,    0,    0,    0,    2,    1,    0,    0,    0,    0,    0,    1,    0,    0,    3,    0,    0,    1,    0,    0,    0,    0,    1,    0,    2,    1,    0,    0,    0,    1,    0,    0,    0,    1,    0,    0,    0,    0,    0,    0,    0,    4,    0,    0,    0,    0,    0,    0,    1,    1,    0,    0,    0,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    3,    0,    1,    0,    1,    1,    1,    0,    0,    0,    0,    0,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    1,    1,    0,    0,    0,    1,    0,    0,   

##### **XGB**

Fit

In [0]:
%%time
import xgboost as xgb
clf4 = xgb.XGBClassifier()
clf4.fit(X_train, y_train)

CPU times: user 18min 44s, sys: 5.35 s, total: 18min 49s
Wall time: 18min 50s


Accuracy

In [0]:
clf4.score(X_test, y_test)

0.16487538681662398

Confusion matrix

In [0]:
y_pred = clf4.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   10,    0,    0,    0,    0,    0,    0,    9,    2,    0,    0,    0,    0,    0,    0,    0,    0,    3,    0,    0,    1,    0,    0,    0,    0,    0,    0,    6,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    2,    0,    4,    0,    0,    0,    2,    0,    0,    0,    0,    2,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    2,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   

##### **LightGBM**

Fit

In [0]:
%%time
import lightgbm as lgb
clf5 = lgb.LGBMClassifier()
clf5.fit(X_train, y_train)

CPU times: user 2min 51s, sys: 684 ms, total: 2min 52s
Wall time: 2min 52s


Accuracy

In [0]:
clf5.score(X_test, y_test)

0.03557049213056933

Confusion matrix

In [0]:
y_pred = clf5.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   7,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   4,   0,   0,   0,   0,   0,   0,   0,   4,   0,   0,   0,   0,   0,   0,   0,   0,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   2,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   4,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   5,   0,   0,   1,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  19,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

##### **MLP**

Fit

In [0]:
%%time
from sklearn import neural_network
clf6 = neural_network.MLPClassifier()
clf6.fit(X_train, y_train)

CPU times: user 7min 19s, sys: 2min 6s, total: 9min 25s
Wall time: 4min 47s


Accuracy

In [0]:
clf6.score(X_test, y_test)

0.07300435896582703

Confusion matrix

In [0]:
y_pred = clf6.predict(X_test)
confusion_matrix(y_test, y_pred)